In [1]:
from sedona.spark import *

In [2]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.6.1,'
           'org.datasyslab:geotools-wrapper:1.6.1-28.2'). \
    getOrCreate()

sedona = SedonaContext.create(config)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-020d62a0-9866-4dcf-83a5-e34529d45d1a;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.6.1 in central
	found org.datasyslab#geotools-wrapper;1.6.1-28.2 in central
:: resolution report :: resolve 126ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.6.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.6.1-28.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      de

In [11]:
buildings_gdf = sedona.read.format("shapefile").option("recursiveFileLookup", "true").load("../ALKIS/*/GebauedeBauwerk.shp").dropDuplicates(["oid"])
buildings_gdf.createOrReplaceTempView("buildings")
nutzung_gdf = sedona.read.format("shapefile").option("recursiveFileLookup", "true").load("../ALKIS/*/NutzungFlurstueck.shp").dropDuplicates(["oid"])
nutzung_gdf.createOrReplaceTempView("usage")

In [4]:
buildings_gdf.count()

2473571

In [5]:
nutzung_gdf.count()

5243004

In [6]:
buildings_gdf.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- oid: string (nullable = true)
 |-- aktualit: string (nullable = true)
 |-- gebnutzbez: string (nullable = true)
 |-- funktion: string (nullable = true)
 |-- gfkzshh: string (nullable = true)
 |-- rellage: string (nullable = true)
 |-- name: string (nullable = true)
 |-- anzahlgs: long (nullable = true)
 |-- gmdschl: string (nullable = true)
 |-- lagebeztxt: string (nullable = true)



In [7]:
nutzung_gdf.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- oid: string (nullable = true)
 |-- aktualit: string (nullable = true)
 |-- nutzart: string (nullable = true)
 |-- bez: string (nullable = true)
 |-- name: string (nullable = true)
 |-- flaeche: decimal(12,2) (nullable = true)
 |-- flstkennz: string (nullable = true)



In [30]:
result_gdf = sedona.sql("""SELECT ST_Intersection(b.geometry, u.geometry) AS geometry, b.oid AS building_oid, b.gebnutzbez, b.gfkzshh, b.name, b.anzahlgs, b.gmdschl, b.lagebeztxt, b.funktion, u.oid AS flur_oid, u.nutzart, u.bez, u.flstkennz
                       FROM buildings b, usage u 
                       WHERE ST_Intersects(b.geometry, u.geometry)"""
                   )

In [39]:
result_gdf.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- building_oid: string (nullable = true)
 |-- gebnutzbez: string (nullable = true)
 |-- gfkzshh: string (nullable = true)
 |-- name: string (nullable = true)
 |-- anzahlgs: long (nullable = true)
 |-- gmdschl: string (nullable = true)
 |-- lagebeztxt: string (nullable = true)
 |-- funktion: string (nullable = true)
 |-- flur_oid: string (nullable = true)
 |-- nutzart: string (nullable = true)
 |-- bez: string (nullable = true)
 |-- flstkennz: string (nullable = true)



In [73]:
result_gdf.createOrReplaceTempView("result")

result2_gdf = sedona.sql("""
                        SELECT ST_MakeValid(geometry), building_oid, gebnutzbez, gfkzshh, name, anzahlgs, gmdschl, lagebeztxt, funktion, flur_oid, nutzart, bez, flstkennz
                        FROM result
                         """)

In [74]:
result2_gdf.show()

24/10/07 20:01:34 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+--------------------+------------------+----------+----------+--------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            geometry|      building_oid|gebnutzbez|   gfkzshh|                name|anzahlgs| gmdschl|          lagebeztxt|            funktion|            flur_oid|             nutzart|                 bez|           flstkennz|
+--------------------+------------------+----------+----------+--------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|POLYGON ((335085....|DEBBAL01000coyoaBL|  Gebaeude|31001_2463|              Garage|       1|12069020|                    |              Garage|DEBBAL010007Eu5HD...|     Strassenverkehr|                    |12156700600255______|
|POLYGON ((322476....|DEBBAL6900027Sq8BL|  Gebaeude|31001_1000|         Wohngebäude|

In [75]:
result2_gdf.count()

24/10/07 20:02:01 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.
24/10/07 20:02:05 ERROR Executor: Exception in task 0.0 in stage 294.0 (TID 3518)
org.apache.spark.sql.sedona_sql.expressions.InferredExpressionException: Exception occurred while evaluating expression ST_Intersection - inputs: [POLYGON ((315056.626 5793716.89, 315053.766 5793719.782, 315058.583 5793724.51, 315058.044 5793725.017, 315061.31 5793728.339, 315061.411 5793728.25, 315063.105 5793729.971, 315060.495 5793732.497, 315061.296 5793733.282, 315056.48 5793738.006, 315063.126 5793744.858, 315073.89 5793734.467, 315066.974 5793727.383, 315064.59 5793725.022, 315061.447 5793721.812, 315060.804 5793721.166, 315056.626 5793716.89)), MULTIPOLYGON (((315050.07 5793723.45, 315056.63 5793716.89, 315053.75 5793719.77, 315050.07 5793723.45)), ((315050.07 5793723.45, 315053.75 5793719.77, 315056.63 5793716.89, 315050.52 5793714.47, 315045.44 5793708.28, 315031.32 5793688.73, 315024.03 5793672

Py4JJavaError: An error occurred while calling o255.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 294.0 failed 1 times, most recent failure: Lost task 0.0 in stage 294.0 (TID 3518) (28dd096dafc7 executor driver): org.apache.spark.sql.sedona_sql.expressions.InferredExpressionException: Exception occurred while evaluating expression ST_Intersection - inputs: [POLYGON ((315056.626 5793716.89, 315053.766 5793719.782, 315058.583 5793724.51, 315058.044 5793725.017, 315061.31 5793728.339, 315061.411 5793728.25, 315063.105 5793729.971, 315060.495 5793732.497, 315061.296 5793733.282, 315056.48 5793738.006, 315063.126 5793744.858, 315073.89 5793734.467, 315066.974 5793727.383, 315064.59 5793725.022, 315061.447 5793721.812, 315060.804 5793721.166, 315056.626 5793716.89)), MULTIPOLYGON (((315050.07 5793723.45, 315056.63 5793716.89, 315053.75 5793719.77, 315050.07 5793723.45)), ((315050.07 5793723.45, 315053.75 5793719.77, 315056.63 5793716.89, 315050.5

In [66]:
result2_gdf = sedona.sql("""
                        SELECT building_oid
                        FROM result
                        WHERE ST_IsValid(geometry, 1) = false;
                         """)

In [67]:
result2_gdf.show()

24/10/07 19:47:31 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.
24/10/07 19:47:35 ERROR Executor: Exception in task 0.0 in stage 272.0 (TID 3284)
org.apache.spark.sql.sedona_sql.expressions.InferredExpressionException: Exception occurred while evaluating expression ST_Intersection - inputs: [POLYGON ((354735.34 5758459.87, 354741.52 5758459.29, 354741.47 5758458.76, 354742.003 5758458.661, 354742.51 5758458.467, 354742.974 5758458.186, 354743.379 5758457.825, 354743.713 5758457.397, 354743.964 5758456.917, 354744.125 5758456.399, 354744.19 5758455.86, 354744.839 5758455.842, 354744.839 5758454.833, 354744.11 5758454.92, 354743.905 5758454.454, 354743.625 5758454.029, 354743.276 5758453.658, 354742.87 5758453.351, 354742.418 5758453.117, 354741.933 5758452.963, 354741.429 5758452.894, 354740.92 5758452.91, 354740.87 5758452.33, 354734.69 5758452.91, 354734.47 5758450.53, 354719.262 5758452.06, 354719.872 5758458.208, 354718.444 5758458.366, 354719.1

Py4JJavaError: An error occurred while calling o242.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 272.0 failed 1 times, most recent failure: Lost task 0.0 in stage 272.0 (TID 3284) (28dd096dafc7 executor driver): org.apache.spark.sql.sedona_sql.expressions.InferredExpressionException: Exception occurred while evaluating expression ST_Intersection - inputs: [POLYGON ((354735.34 5758459.87, 354741.52 5758459.29, 354741.47 5758458.76, 354742.003 5758458.661, 354742.51 5758458.467, 354742.974 5758458.186, 354743.379 5758457.825, 354743.713 5758457.397, 354743.964 5758456.917, 354744.125 5758456.399, 354744.19 5758455.86, 354744.839 5758455.842, 354744.839 5758454.833, 354744.11 5758454.92, 354743.905 5758454.454, 354743.625 5758454.029, 354743.276 5758453.658, 354742.87 5758453.351, 354742.418 5758453.117, 354741.933 5758452.963, 354741.429 5758452.894, 354740.92 5758452.91, 354740.87 5758452.33, 354734.69 5758452.91, 354734.47 5758450.53,

In [50]:
proj = """{"$schema": "https://proj.org/schemas/v0.7/projjson.schema.json","type": "ProjectedCRS","name": "ETRS89 / UTM zone 33N","base_crs": {"name": "ETRS89","datum_ensemble": {"name": "European Terrestrial Reference System 1989 ensemble","members": [{"name": "European Terrestrial Reference Frame 1989"},{"name": "European Terrestrial Reference Frame 1990"},{"name": "European Terrestrial Reference Frame 1991"},{"name": "European Terrestrial Reference Frame 1992"},{"name": "European Terrestrial Reference Frame 1993"},{"name": "European Terrestrial Reference Frame 1994"},{"name": "European Terrestrial Reference Frame 1996"},{"name": "European Terrestrial Reference Frame 1997"},{"name": "European Terrestrial Reference Frame 2000"},{"name": "European Terrestrial Reference Frame 2005"},{"name": "European Terrestrial Reference Frame 2014"}],"ellipsoid": {"name": "GRS 1980","semi_major_axis": 6378137,"inverse_flattening": 298.257222101},"accuracy": "0.1"},"coordinate_system": {"subtype": "ellipsoidal","axis": [{"name": "Geodetic latitude","abbreviation": "Lat","direction": "north","unit": "degree"},{"name": "Geodetic longitude","abbreviation": "Lon","direction": "east","unit": "degree"}]},"id": {"authority": "EPSG","code": 4258}},"conversion": {"name": "UTM zone 33N","method": {"name": "Transverse Mercator","id": {"authority": "EPSG","code": 9807}},"parameters": [{"name": "Latitude of natural origin","value": 0,"unit": "degree","id": {"authority": "EPSG","code": 8801}},{"name": "Longitude of natural origin","value": 15,"unit": "degree","id": {"authority": "EPSG","code": 8802}},{"name": "Scale factor at natural origin","value": 0.9996,"unit": "unity","id": {"authority": "EPSG","code": 8805}},{"name": "False easting","value": 500000,"unit": "metre","id": {"authority": "EPSG","code": 8806}},{"name": "False northing","value": 0,"unit": "metre","id": {"authority": "EPSG","code": 8807}}]},"coordinate_system": {"subtype": "Cartesian","axis": [{"name": "Easting","abbreviation": "E","direction": "east","unit": "metre"},{"name": "Northing","abbreviation": "N","direction": "north","unit": "metre"}]},"scope": "Engineering survey, topographic mapping.","area": "Europe between 12°E and 18°E: Austria; Denmark - offshore and offshore; Germany - onshore and offshore; Norway including Svalbard - onshore and offshore.","bbox": {"south_latitude": 46.4,"west_longitude": 12,"north_latitude": 84.42,"east_longitude": 18.01},"id": {"authority": "EPSG","code": 25833}}"""

(result2_gdf.write
 .format("geoparquet")
 .mode("overwrite")
 .option("geoparquet.version", "1.0.0")
 .option("geoparquet.crs", proj)
 .save("sedona_data.parquet")
)

24/10/07 19:28:51 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.
24/10/07 19:28:57 ERROR Utils: Aborting task                      (0 + 16) / 32]
org.locationtech.jts.geom.TopologyException: side location conflict [ (404524.85, 5795432.82, NaN) ]
	at org.locationtech.jts.geomgraph.EdgeEndStar.propagateSideLabels(EdgeEndStar.java:289)
	at org.locationtech.jts.geomgraph.EdgeEndStar.computeLabelling(EdgeEndStar.java:128)
	at org.locationtech.jts.operation.relate.RelateComputer.labelNodeEdges(RelateComputer.java:325)
	at org.locationtech.jts.operation.relate.RelateComputer.computeIM(RelateComputer.java:125)
	at org.locationtech.jts.operation.relate.RelateOp.getIntersectionMatrix(RelateOp.java:109)
	at org.locationtech.jts.operation.relate.RelateOp.relate(RelateOp.java:54)
	at org.locationtech.jts.geom.Geometry.relate(Geometry.java:1035)
	at org.locationtech.jts.geom.Geometry.intersects(Geometry.java:767)
	at org.apache.sedona.core.spatialOperator.Spatia

Py4JJavaError: An error occurred while calling o183.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 217.0 failed 1 times, most recent failure: Lost task 7.0 in stage 217.0 (TID 2686) (28dd096dafc7 executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/app/geospeed/sedona_data.parquet.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:788)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:888)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:888)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadC

In [70]:
sedona

In [71]:
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
